<a href="https://colab.research.google.com/github/Ravgeer/sparkscon2024/blob/main/Sparkscon2024_ComputerVision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
# **1. Clone Data Directory, Pretrained Models, Scripts and Augmentations**


In [ ]:
!git clone https://github.com/Ravgeer/sparkscon2024

---
# **2. Clone and Install YOLO and download pre-trained models for Classification**


In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

import torch
import utils
display = utils.notebook_init()  # checks

In [ ]:
from utils.downloads import attempt_download
p5 = ['n','s','l','m','x']

cls = [f'{x}-cls' for x in p5]

for x in cls:
  attempt_download(f'weights/yolov5{x}.pt')

---
# **3. Generate Augmentations**


In [ ]:
### Python Script for image scraping from Weblinks
!python /content/yolov5/sparkscon2024/scripts/scrape_images.py 'https://www.shutterstock.com/search/granny-smith?image_type=photo' '/content/scrape_images/apples/granny_smith_new' --delay 1.0


In [ ]:
### Python Script for image Augmentation from data Source
!python /content/yolov5/sparkscon2024/scripts/augmented_images.py '/content/sparkscon2024/scrape_images/apples/train/braeburn' '/content/sparkscon2024/scrape_images/apples/train/braeburn_new' --num_augmentations 2

---
# **4. Image Classification Training**




In [ ]:
#Connect G-Drive to exchange data
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
### set optimal_batch_size
import torch
from utils.torch_utils import select_device
from utils.autobatch import check_train_batch_size

def find_optimal_batch(weights, imgsz=640, amp=True):
    # Select the device (GPU or CPU)
    device = select_device('')
    print(f"selected_device: {device}")
    # Load the model
    model = torch.load(weights, map_location=device)['model'].float()  # Load FP32 model
    model.fuse()  # Fuse Conv2d + BatchNorm2d layers
    if amp:
        model.half()  # Convert model to FP16 if amp (automatic mixed precision) is enabled

    # Find optimal batch size
    return check_train_batch_size(model, imgsz=imgsz, amp=amp)

if __name__ == '__main__':
    weights = '/content/yolov5/weights/yolov5l-cls.pt'  # Path to your weights file
    imgsz = 640  # Image size
    amp = True  # Automatic Mixed Precision (AMP)

    optimal_batch_size = find_optimal_batch(weights, imgsz, amp)
    print(f"Optimal batch size: {optimal_batch_size}")


In [ ]:
## optimal_batch_size ##
!python -W ignore classify/train.py --batch {optimal_batch_size} --epochs 2 --data  /content/sparkscon2024/scrape_images/apples/ --model yolov5s-cls.pt --name yolov5s_results


In [ ]:
### User defined batch size ##
!python -W ignore classify/train.py --img 640 --batch 16 --epochs 2 --data  /content/sparkscon2024/scrape_images/apples/ --model yolov5l-cls.pt --name yolov5l_results


In [ ]:
#### We can copy our trained models in Google drive with this command
!cp -r /content/yolov5/runs/train-cls/  /content/drive/MyDrive/

---
# **5. Test Custom Model using Test Images**


In [ ]:
!python classify/predict.py --weights /content/sparkscon2024/models/weights/best.pt --source /content/sparkscon2024/scrape_images/apples/test/jonagored/pear-amidst-red-jonagored-apples-260nw-2389319923.jpg --save-txt

### yolo_models_path:     /content/yolov5/runs/train-cls/yolov5l_results/weights/best.pt
### standard yolo model:  yolov5l-cls.pt

---
# **6. Export model**


In [ ]:
!python export.py --weights /content/yolov5/runs/train-cls/yolov5l_results/weights/best.pt --include onnx

---
# **7. Load Model**


In [ ]:
from ultralytics import YOLO

# Load a model
# model = YOLO('yolov8n.pt')  # load an official model
model = YOLO('/content/yolov5/runs/train-cls/yolov5l_results/weights/best.pt')  # load a custom trained model

# Export the model
model.export(format='onnx')

---
# **Archive**


In [ ]:
#Connect G-Drive to exchange data
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
## required for Autobatch
%cd ..
!git clone https://github.com/ultralytics/ultralytics.git

In [ ]:
###!python /content/yolov5/ultralytics/ultralytics/utils/autobatch.py

In [ ]:
%pwd

In [ ]:
!rm -r '/content/scrape_images'